Again, before start, let's import a few Python modules.

This time we will focus on a new module of `kongming.memory`.

In [1]:
from kongming import api, hv, memory 
from kongming.hv import helpers

d0 = hv.d0()

GOMAXPROCS set to 128 for this invocation.


Like we did before, we generate some primitive random hypervectors.

In [2]:
primitives={chr(letter): d0.new_sparse_segmented_from_seed_word(api.MODEL_1M_10BIT, chr(letter)) for letter in range(ord('a'), ord('z')+1)}

str(primitives['a']), str(primitives['k'])

("ss(m=MODEL_1M_10BIT, hash=0x40fa56a18e54f291, 'a', exp=1)",
 "ss(m=MODEL_1M_10BIT, hash=0xabcc3adf6707dbf2, 'k', exp=1)")

# Working with CodePickers

`memory.CodePickers`, as its name suggests, are pre-defined logic that selects and picks codes. For example, we can pick constituents from a composite code representing a sequence, or we can pick nearby registered codes from a given probe.

In [8]:
world = d0.new_sequence(primitives['w'], primitives['o'], primitives['r'], primitives['l'], primitives['d']).compose(1)

helpers.to_native_hbp(world.proto_load())

hint: SEQUENCE
model: MODEL_1M_10BIT
stable_hash: 7016629907439205749
sequence {
  seed: 1
  contains {
    members {
      sparse_segmented {
        seed_word: "w"
      }
    }
    members {
      sparse_segmented {
        seed_word: "o"
      }
    }
    members {
      sparse_segmented {
        seed_word: "r"
      }
    }
    members {
      sparse_segmented {
        seed_word: "l"
      }
    }
    members {
      sparse_segmented {
        seed_word: "d"
      }
    }
  }
}

The above `world` itself is a composed `hv.Sequence` instance, and you can think of it as a sequence code for the word `world`.

To recover individual members (`w`, `o`, `r`, `l` and `d`), we can use a instance `mp` of type `memory.sequence_members`.

In [12]:
mp = memory.sequence_members(api.MODEL_1M_10BIT, memory.code_as_picker(world), 0, 0)

helpers.to_native_picker(mp.proto_load())

seq_members_of {
  model: MODEL_1M_10BIT
  sequence {
    code {
      hint: SEQUENCE
      model: MODEL_1M_10BIT
      stable_hash: 7016629907439205749
      sequence {
        seed: 1
        contains {
          members {
            sparse_segmented {
              seed_word: "w"
            }
          }
          members {
            sparse_segmented {
              seed_word: "o"
            }
          }
          members {
            sparse_segmented {
              seed_word: "r"
            }
          }
          members {
            sparse_segmented {
              seed_word: "l"
            }
          }
          members {
            sparse_segmented {
              seed_word: "d"
            }
          }
        }
      }
    }
  }
}

It has to be noted that the input for `memory.sequence_members` doesn't have to be the fully composed sequence code. Even with a flattened code (`world.core()`), the associative index should enable this code picker to do the right job. 

In [13]:
mp0 = memory.sequence_members(api.MODEL_1M_10BIT, memory.code_as_picker(world.core()), 0, 0)

helpers.to_native_picker(mp0.proto_load())

seq_members_of {
  model: MODEL_1M_10BIT
  sequence {
    code {
      model: MODEL_1M_10BIT
      stable_hash: 7016629907439205749
      sparse_segmented {
        offsets: "\244\030\004)z\327\231\\O+.\320{\010\235\216q\336,\263yG\274\027\367V=5*\334\333\207.\251)a~\353\r\331.2\304M:tp\361\014ey\336\020\213\334\370\2270\263n\214i\317\220n\244\212\205j\223\005\316M\362\000\331\362\321\320\256\265\254\301{\tC\275\355\253\324e\301\302\230#\244\237#\241\'&\36431\251A\250|hYu\273>C\267a\372M\262\375\353\355\266C\351\177#2\276X\306-\030\307YD\230\004H\323\201\273\026\374\247w\034\247V\013\323\346\004\317\336 \3463\353\035\n\206\321\033\264B\312\300\275\005`\251e)\005 T\031i\226\r\235\036[@\362\375\002tW\177\356\026\216\306\337,\277\206\276\214l=\230D\n\000\256\354\210n~\247\020D.\2146kJ\225\325\243s\311\212\350\222\252\3462W\275\236t\203\225\306&\016n\340N\300g\244\275\214:\2328\353\305m\\\351\2278\004\206>cAB\202\002k\341\225\277D\377\215\216\350\2679\220\317\031\372\210\371B\353D\374\000\

Notice the sequence codes from both `mp` and `mp1` are identical (from their `stable_hash`), and should return the same results.

# Interacts with memory

Once we have a good understanding of how hypervectors and their interactions, the next logical step is to store and retrieve them from a memory. 

Memory is a component that provides efficient storage and near-neighbor search capability for hypervectors. This is analogous to the vector database as discussed in industry. 